In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv 
load_dotenv()
import yaml
import os
from crewai import Agent, Task, Crew

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': r'src\portfolio_assistant\config\agents.yaml',
    'tasks': r'src\portfolio_assistant\config\tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [ ]:
agents_config

In [ ]:
tasks_config

In [ ]:
from pydantic import BaseModel
from typing import List

class WorkExperience(BaseModel):
    job_title: str
    company: str
    start_date: str
    end_date: str
    description: str


class Education(BaseModel):
    degree: str
    institution: str
    graduation_year: str


class Project(BaseModel):
    name: str
    description: str
    technologies: List[str]


class Certification(BaseModel):
    name: str
    issuing_organization: str


class CVInfo(BaseModel):
    work_experience: List[WorkExperience]
    education: List[Education]
    skills: List[str]
    projects: List[Project]
    certifications: List[Certification]


class CVReference(BaseModel):
    section: str  # Example: "work_experience", "skills", "education"
    details: str  # Summary of relevant data from `CVInfo`


class VisitorResponse(BaseModel):
    question: str
    answer: str
    references: List[CVReference]  # Where the answer is derived from

In [ ]:
from crewai_tools import PDFSearchTool
pdf_search_tool = PDFSearchTool(pdf="knowledge/CV_Assia_Chafi_2025.pdf")
pdf_search_tool.description = "Search the CV PDF with a text query to find relevant information"
# Now create your agents
from crewai import Agent, Task
researcher = Agent(config=agents_config["resume_assistant"], verbose=True)
reporting_analyst = Agent(config=agents_config["reporting_analyst"], verbose=True)

research_task = Task(
    description=tasks_config["extract_cv_info"]["description"],
    expected_output=tasks_config["extract_cv_info"]['expected_output'],
    output_pydantic=CVInfo,
    tools=[pdf_search_tool],
    agent = researcher
)

reporting_task = Task(
    description=tasks_config["reporting_task"]['description'],
    expected_output=tasks_config["reporting_task"]['expected_output'],
    output_pydantic=VisitorResponse,
    output_file="report.md",
    agent=reporting_analyst
)

In [ ]:
# Creating Crew
crew = Crew(
  agents=[
    researcher,
    reporting_analyst
  ],
  tasks=[
    research_task,
    reporting_task
  ],
  verbose=True
)

In [ ]:
crew

In [ ]:
# The given Python dictionary
inputs = {
  'question': "What are your main skills?",
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

In [ ]:
result['answer']
